## _*Using Algorithm Concatenation in Qiskit Aqua*_

This notebook demonstrates how to use the `Qiskit Aqua` library to realize algorithm concatenation. In particular, we experiment with chaining the executions of VQE and IQPE by first running VQE and then preparing IQPE's initial state using the variational form as produced by VQE upon its termination.

In [1]:
import numpy as np
from qiskit_aqua.input import get_input_instance
from qiskit_aqua import Operator, run_algorithm, get_algorithm_instance, get_variational_form_instance, get_optimizer_instance
from qiskit_aqua.algorithms.components.initial_states.varformbased import VarFormBased

Here an Operator instance is created for our Hamiltonian, for which we are going to estimation the ground energy level. In this case the paulis are from a previously computed Hamiltonian for simplicity.

In [2]:
pauli_dict = {
    'paulis': [{"coeff": {"imag": 0.0, "real": -1.052373245772859}, "label": "II"},
              {"coeff": {"imag": 0.0, "real": 0.39793742484318045}, "label": "ZI"},
              {"coeff": {"imag": 0.0, "real": -0.39793742484318045}, "label": "IZ"},
              {"coeff": {"imag": 0.0, "real": -0.01128010425623538}, "label": "ZZ"},
              {"coeff": {"imag": 0.0, "real": 0.18093119978423156}, "label": "XX"}
              ]
}

qubitOp = Operator.load_from_dict(pauli_dict)

We can now use the Operator without regard to how it was created. First we will use the ExactEigensolver to compute the reference ground energy level.

In [3]:
algorithm_cfg = {
    'name': 'ExactEigensolver',
}

params = {
    'algorithm': algorithm_cfg
}
algo_input = get_input_instance('EnergyInput')
algo_input.qubit_op = qubitOp
result_reference = run_algorithm(params,algo_input)
print('The reference ground energy level is {}.'.format(result_reference['energy']))

The reference ground energy level is -1.857275030202382.


Having established the reference ground energy, we next carry on with our experiment. First we configure a VQE algorithm instance. The idea is that we can set an termination condition such that the VQE instance returns rather quickly with a rough estimation result.

In [4]:
np.random.seed(0)

var_form_depth = 3
var_form = get_variational_form_instance('RYRZ')
var_form.init_args(algo_input.qubit_op.num_qubits, var_form_depth)

spsa_max_trials=10
optimizer = get_optimizer_instance('SPSA')
optimizer.init_args(max_trials=spsa_max_trials)

vqe_mode = 'paulis'
vqe = get_algorithm_instance('VQE')
vqe.setup_quantum_backend(backend='qasm_simulator')
vqe.init_args(algo_input.qubit_op, vqe_mode, var_form, optimizer)

result_vqe = vqe.run()
print('VQE estimated the ground energy to be {}.'.format(result_vqe['energy']))

VQE estimated the ground energy to be -1.7740092203003834.


As previously indicated, the energy estimation result is rather rough--it is far from being an acceptable final estimation figure. But, it is close enough such that the accompanying variational form might be a reasonably good approximation to the ground eigenstate, which means the corresponding wave function can serve as the initial state for the IQPE execution that follows. We next prepare such an initial state.

In [5]:
state_in = VarFormBased()
state_in.init_args(var_form, result_vqe['opt_params'])

With the VQE-generated quantum state wave function serving as the chaining piece and prepared as initial state, we now go ahead with configuring and running an IQPE instance.

In [6]:
iqpe = get_algorithm_instance('IQPE')
iqpe.setup_quantum_backend(backend='qasm_simulator', shots=100)

num_time_slices = 50
num_iterations = 11

iqpe.init_args(
    algo_input.qubit_op, state_in, num_time_slices, num_iterations,
    paulis_grouping='random',
    expansion_mode='suzuki',
    expansion_order=2,
)

result_iqpe = iqpe.run()
print("Continuing with VQE's result, IQPE estimated the ground energy to be {}.".format(result_iqpe['energy']))

Continuing with VQE's result, IQPE estimated the ground energy to be -1.8531516030612387.


As seen, the final ground energy estimation as produced by IQPE is much more accurate that the intermediate result as produced by VQE.